# Graph Analysis

### Connect with database

In [20]:
from neo4j import GraphDatabase

URI = "neo4j://localhost:7687"
NAME_DB = "movies"

driver = GraphDatabase.driver(URI)

driver.verify_connectivity()

print("Successfully connected to Neo4j")

Successfully connected to Neo4j


In [21]:
session = driver.session()

if [NAME_DB] in session.run("SHOW DATABASE yield name;").values():
    print("Database already exists")
else:
    session.run(f"CREATE DATABASE {NAME_DB};")
    print("Database created")
session = driver.session(database=NAME_DB)

Database already exists


### Create constraints

In [22]:
# TODO: maybe add constraints for id for edges
session.run(
    """
    CREATE CONSTRAINT UniqueMovieID IF NOT EXISTS 
    FOR (m:Movie) 
    REQUIRE m.id IS unique;
    """
)
session.run(
    """
    CREATE CONSTRAINT UniqueUserID IF NOT EXISTS 
    FOR (u:User) 
    REQUIRE u.id IS unique;
    """
)
# TODO: add constraints for relationships

### Add nodes to database

In [23]:
session.run(
    """
    LOAD CSV WITH HEADERS 
    FROM "file:///users.csv" AS row
    MERGE (u:User {id: toInteger(row.userId)});
    """
)
session.run(
    """
    LOAD CSV WITH HEADERS 
    FROM "file:///movies.csv" AS row
    MERGE (m:Movie {id: toInteger(row.movieId)})
    SET m.title = row.title,
        m.genres = split(row.genres, '|');
    """
)

In [24]:
# TODO: Create indexes with sense
# session.run(
#     """
#     CREATE INDEX MovieTitleIndex FOR (m:Movie) ON (m.title);
#     """
# )
# session.run(
#     """
#     CREATE INDEX MovieGenresIndex FOR (m:Movie) ON (m.genres);
#     """
# )

In [25]:
# TODO:Switch to APOC

session.run(
    """
    LOAD CSV WITH HEADERS FROM "file:///ratings.csv" AS row
    MATCH (m:Movie {id: toInteger(row.movieId)})
    MATCH (u:User {id: toInteger(row.userId)})
    MERGE (u)-[r:RATED]->(m)
    SET r.rating = toFloat(row.rating);
    """
)

session.run(
    """
    LOAD CSV WITH HEADERS FROM "file:///tags.csv" AS row
    MATCH (m:Movie {id: toInteger(row.movieId)})
    MATCH (u:User {id: toInteger(row.userId)})
    MERGE (u)-[t:TAGGED]->(m)
    SET t.rating = row.tag;
    """
)